In [51]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wbgapi as wb


In [52]:
# Load data
df = pd.read_excel('eu_cyberattacks.xlsx')

In [53]:
# Show first 5 rows
df.head()

,country,NATO Member,International Cooperation,Incident Response Capabilities,Cyber Crime,Training and Education,CIIP,Citizen's Awarness,Incident Report Mechanism,Baseline Security Requirements,...,Risk Assessment,Incentives to private sector to invest,Trusted Information-sharing mechanism,Establish and implement policies and regulation,Clear Governance Structure,Cybersecurity of the supply chain,CSIRTs,Military CSIRTs,Cyber Exercises under CSDP,Average number of cyber incidents
0,Austria,No,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,10,0,0,NaN
1,Belgium,Yes,1,1,1,1,0,1,1,0,...,0,0,0,0,0,0,11,0,1,NaN
2,Bulgaria,Yes,1,1,1,1,0,1,1,0,...,0,0,0,0,0,0,1,0,1,NaN
3,Croatia,Yes,1,1,1,0,0,0,1,1,...,0,0,0,0,0,0,2,0,1,NaN
4,Cyprus,Yes,1,1,1,1,1,1,0,0,...,1,1,0,1,1,0,3,0,2,NaN


In [54]:
# Create another dataframe with Log GDP per capita from World Bank only for 2022 and EU countries
df_gdp = wb.data.DataFrame('NY.GDP.PCAP.CD', time=2022, labels=True, skipAggs=True)

In [55]:
print(df_gdp.head())

                       Country  NY.GDP.PCAP.CD
economy                                       
ZWE                   Zimbabwe     1266.996031
ZMB                     Zambia     1487.907764
YEM                Yemen, Rep.      676.928385
PSE         West Bank and Gaza     3789.327966
VIR      Virgin Islands (U.S.)             NaN


In [56]:
# Mapping dictionary to handle country name differences
country_mapping = {
    'Czech Republic': 'Czechia',
    'Slovakia': 'Slovak Republic'
}

# Apply the mapping to the 'country' column in df
df['country'] = df['country'].replace(country_mapping)

# List of countries from df
countries_list = df['country']

In [57]:
# Filter df_gdp to include only matching countries
filtered_df_gdp = df_gdp[df_gdp['Country'].isin(countries_list)]

# Display the filtered DataFrame
print(filtered_df_gdp)


                 Country  NY.GDP.PCAP.CD
economy                                 
SWE               Sweden    55873.220811
ESP                Spain    29350.168521
SVN             Slovenia    29457.402844
SVK      Slovak Republic    21258.114135
ROU              Romania    15892.118526
PRT             Portugal    24274.516582
POL               Poland    18321.280890
NLD          Netherlands    55985.403426
MLT                Malta    33940.949596
LUX           Luxembourg   126426.089934
LTU            Lithuania    24826.791037
LVA               Latvia    21851.105201
ITA                Italy    34157.990122
IRL              Ireland   104038.946076
HUN              Hungary    18463.208525
GRC               Greece    20732.052219
DEU              Germany    48432.455873
FRA               France    40963.837337
FIN              Finland    50536.624467
EST              Estonia    28332.629092
DNK              Denmark    66983.132002
CZE              Czechia    27638.373276
CYP             

In [58]:
import math

# Merge the GDP per capita values from df_gdp into df based on the "Country" column
merged_df = df.merge(df_gdp[['Country', 'NY.GDP.PCAP.CD']], left_on='country', right_on='Country', how='left')

# Rename the merged column to 'log_GDP_percapita' and take the logarithm
merged_df['log_GDP_percapita'] = merged_df['NY.GDP.PCAP.CD'].apply(lambda x: None if pd.isnull(x) else round(math.log(x), 2))

# Drop the redundant 'Country' column
merged_df.drop(columns=['Country'], inplace=True)

# Display the merged DataFrame with the new column
print(merged_df)

            country NATO Member  International Cooperation  \
0           Austria          No                          1   
1           Belgium         Yes                          1   
2          Bulgaria         Yes                          1   
3           Croatia         Yes                          1   
4            Cyprus         Yes                          1   
5           Czechia         Yes                          1   
6           Denmark         Yes                          1   
7           Estonia         Yes                          1   
8           Finland         Yes                          1   
9            France         Yes                          1   
10          Germany         Yes                          1   
11           Greece         Yes                          1   
12          Hungary         Yes                          1   
13          Ireland         Yes                          1   
14            Italy         Yes                          1   
15      

In [59]:
# Display the merged DataFrame with the new column
print(merged_df)

            country NATO Member  International Cooperation  \
0           Austria          No                          1   
1           Belgium         Yes                          1   
2          Bulgaria         Yes                          1   
3           Croatia         Yes                          1   
4            Cyprus         Yes                          1   
5           Czechia         Yes                          1   
6           Denmark         Yes                          1   
7           Estonia         Yes                          1   
8           Finland         Yes                          1   
9            France         Yes                          1   
10          Germany         Yes                          1   
11           Greece         Yes                          1   
12          Hungary         Yes                          1   
13          Ireland         Yes                          1   
14            Italy         Yes                          1   
15      

In [60]:
# Filter and print countries with NaN values in the "log_GDP_percapita" column
countries_with_nan_gdp = merged_df[merged_df['log_GDP_percapita'].isna()]['country']

# Print the list of countries
print(countries_with_nan_gdp)

Series([], Name: country, dtype: object)


In [61]:
# Delete NY.GDP.PCAP.CD column
del merged_df['NY.GDP.PCAP.CD']

## Log of ICT service exports ##

In [62]:
ict = wb.data.DataFrame('BX.GSR.CCIS.CD', time=2021, labels=True, skipAggs=True)

# Filter df_gdp to include only matching countries
filtered_df_ict = ict[ict['Country'].isin(countries_list)]

# Merge the GDP per capita values from df_gdp into df based on the "Country" column
merged_df = merged_df.merge(filtered_df_ict[['Country', 'BX.GSR.CCIS.CD']], left_on='country', right_on='Country', how='left')

# Rename the merged column to 'log_GDP_percapita' and take the logarithm
merged_df['log_ICT_exports'] = merged_df['BX.GSR.CCIS.CD'].apply(lambda x: None if pd.isnull(x) else round(math.log(x), 2))

# Drop the redundant 'Country' column
merged_df.drop(columns=['Country'], inplace=True)

del merged_df["BX.GSR.CCIS.CD"]

# Display the merged DataFrame with the new column
print(merged_df)



            country NATO Member  International Cooperation  \
0           Austria          No                          1   
1           Belgium         Yes                          1   
2          Bulgaria         Yes                          1   
3           Croatia         Yes                          1   
4            Cyprus         Yes                          1   
5           Czechia         Yes                          1   
6           Denmark         Yes                          1   
7           Estonia         Yes                          1   
8           Finland         Yes                          1   
9            France         Yes                          1   
10          Germany         Yes                          1   
11           Greece         Yes                          1   
12          Hungary         Yes                          1   
13          Ireland         Yes                          1   
14            Italy         Yes                          1   
15      

## Log of scientific and technical journal ##

In [63]:
sci = wb.data.DataFrame('IP.JRN.ARTC.SC', time=2020, labels=True, skipAggs=True)

# Filter df_ict to include only matching countries
filtered_df_sci = sci[sci['Country'].isin(countries_list)]

# Merge the ICT exports values from df_ict into merged_df based on the "Country" column
merged_df = merged_df.merge(filtered_df_sci[['Country', 'IP.JRN.ARTC.SC']], left_on='country', right_on='Country', how='left')

# Rename the merged column to 'log_ICT_exports' and take the logarithm
merged_df['log_Scientific_and_Technical_Journal'] = merged_df['IP.JRN.ARTC.SC'].apply(lambda x: None if pd.isnull(x) else round(math.log(x), 2))

# Drop the redundant 'Country' column
merged_df.drop(columns=['Country'], inplace=True)

del merged_df["IP.JRN.ARTC.SC"]

# Display the merged DataFrame with the new column
print(merged_df)

            country NATO Member  International Cooperation  \
0           Austria          No                          1   
1           Belgium         Yes                          1   
2          Bulgaria         Yes                          1   
3           Croatia         Yes                          1   
4            Cyprus         Yes                          1   
5           Czechia         Yes                          1   
6           Denmark         Yes                          1   
7           Estonia         Yes                          1   
8           Finland         Yes                          1   
9            France         Yes                          1   
10          Germany         Yes                          1   
11           Greece         Yes                          1   
12          Hungary         Yes                          1   
13          Ireland         Yes                          1   
14            Italy         Yes                          1   
15      

In [64]:
reverse_country_mapping = {
    'Czechia': 'Czech Republic',
    'Slovak Republic': 'Slovakia'
}

# Apply the reverse mapping to the 'country' column in df
merged_df['country'] = merged_df['country'].replace(reverse_country_mapping)

## Defense spending ##

In [65]:
defence = pd.read_excel("defense_exp_2021.xlsx")
defence.head()


,country,Total Defence Expenditure per capita,Defence R&D Expenditure (millions)
0,Austria,368,"7,5"
1,Belgium,468,"17,2"
2,Bulgaria,156,"4,8"
3,Croatia,296,"0,6"
4,Cyprus,509,"0,3"


In [66]:
# Merge df_defence with merged_df based on the "country" column
merged_df = merged_df.merge(defence, on='country', how='left')

# Rename the columns to keep only one set of values
merged_df.rename(columns={
    'Total Defence Expenditure per capita_x': 'Total Defence Expenditure per capita',
    'Defence R&D Expenditure (millions)_x': 'Defence R&D Expenditure (millions)'
}, inplace=True)

In [67]:
print(merged_df)

           country NATO Member  International Cooperation  \
0          Austria          No                          1   
1          Belgium         Yes                          1   
2         Bulgaria         Yes                          1   
3          Croatia         Yes                          1   
4           Cyprus         Yes                          1   
5   Czech Republic         Yes                          1   
6          Denmark         Yes                          1   
7          Estonia         Yes                          1   
8          Finland         Yes                          1   
9           France         Yes                          1   
10         Germany         Yes                          1   
11          Greece         Yes                          1   
12         Hungary         Yes                          1   
13         Ireland         Yes                          1   
14           Italy         Yes                          1   
15          Latvia      

## Average number of cyberattacks ##

In [68]:
# Load attacks.csv
attacks = pd.read_csv('attacks.csv')

In [69]:
# Merge attacks.csv into merged_df based on country
merged_df = merged_df.merge(attacks, on='country', how='left')
merged_df.head()

,country,NATO Member,International Cooperation,Incident Response Capabilities,Cyber Crime,Training and Education,CIIP,Citizen's Awarness,Incident Report Mechanism,Baseline Security Requirements,...,Military CSIRTs,Cyber Exercises under CSDP,Average number of cyber incidents,log_GDP_percapita,log_ICT_exports,log_Scientific_and_Technical_Journal,Total Defence Expenditure per capita,Defence R&D Expenditure (millions),count,average_attacks_standardized
0,Austria,No,1,1,1,1,1,0,1,1,...,0,0,NaN,10.86,23.00,9.53,368.0,"7,5",11.0,0.647059
1,Belgium,Yes,1,1,1,1,0,1,1,0,...,0,1,NaN,10.81,23.61,9.74,468.0,"17,2",19.0,1.117647
2,Bulgaria,Yes,1,1,1,1,0,1,1,0,...,0,1,NaN,9.53,21.72,8.32,156.0,"4,8",11.0,0.647059
3,Croatia,Yes,1,1,1,0,0,0,1,1,...,0,1,NaN,9.82,20.98,8.48,296.0,"0,6",2.0,0.117647
4,Cyprus,Yes,1,1,1,1,1,1,0,0,...,0,2,NaN,10.35,22.26,7.30,509.0,"0,3",4.0,0.235294


In [70]:
merged_df.columns

Index(['country', 'NATO Member', 'International Cooperation',
       'Incident Response Capabilities', 'Cyber Crime',
       'Training and Education', 'CIIP', 'Citizen's Awarness',
       'Incident Report Mechanism', 'Baseline Security Requirements',
       'Cybersecurity  exercises', 'R&D', 'National Cyber Contigency Plans',
       'Cooperatioon between Public Agencies', 'Public-Private Partnership',
       'Balance Security-Privacy', 'Information Security Standards',
       'Risk Assessment', 'Incentives to private sector to invest ',
       'Trusted Information-sharing  mechanism',
       'Establish and implement policies and regulation',
       'Clear Governance Structure', 'Cybersecurity of the supply chain',
       'CSIRTs', 'Military CSIRTs', 'Cyber Exercises under CSDP ',
       ' Average number of cyber incidents', 'log_GDP_percapita',
       'log_ICT_exports', 'log_Scientific_and_Technical_Journal',
       '    Total Defence Expenditure per capita',
       '    Defence R&D Ex

In [71]:
# Delete Average number of cyber incidents and count columns
del merged_df[' Average number of cyber incidents']
del merged_df['count']

# Display the first five rows of merged_df
print(merged_df.head())

    country NATO Member  International Cooperation  \
0   Austria          No                          1   
1   Belgium         Yes                          1   
2  Bulgaria         Yes                          1   
3   Croatia         Yes                          1   
4    Cyprus         Yes                          1   

   Incident Response Capabilities  Cyber Crime  Training and Education  CIIP  \
0                               1            1                       1     1   
1                               1            1                       1     0   
2                               1            1                       1     0   
3                               1            1                       0     0   
4                               1            1                       1     1   

   Citizen's Awarness  Incident Report Mechanism  \
0                   0                          1   
1                   1                          1   
2                   1                 

In [72]:
# Pre-processing
# List of column names you want to check
columns_to_rename = [
    'Cybersecurity  exercises',
    'Incentives to private sector to invest ',
    'Trusted Information-sharing  mechanism',
    'Cyber Exercises under CSDP ',
    '    Total Defence Expenditure per capita',
    '    Defence R&D Expenditure (millions)',
    ' Average number of cyber incidents'
]


# List of corrected column names
corrected_columns = [
    'Cybersecurity exercises',
    'Incentives to private sector to invest',
    'Trusted Information-sharing mechanism',
    'Cyber Exercises under CSDP',
    'Total Defence Expenditure per capita',
    'Defence R&D Expenditure (millions)',
    'Average number of cyber incidents'
]

merged_df.rename(columns=dict(zip(columns_to_rename, corrected_columns)), inplace=True)

In [73]:
merged_df.columns

Index(['country', 'NATO Member', 'International Cooperation',
       'Incident Response Capabilities', 'Cyber Crime',
       'Training and Education', 'CIIP', 'Citizen's Awarness',
       'Incident Report Mechanism', 'Baseline Security Requirements',
       'Cybersecurity exercises', 'R&D', 'National Cyber Contigency Plans',
       'Cooperatioon between Public Agencies', 'Public-Private Partnership',
       'Balance Security-Privacy', 'Information Security Standards',
       'Risk Assessment', 'Incentives to private sector to invest',
       'Trusted Information-sharing mechanism',
       'Establish and implement policies and regulation',
       'Clear Governance Structure', 'Cybersecurity of the supply chain',
       'CSIRTs', 'Military CSIRTs', 'Cyber Exercises under CSDP',
       'log_GDP_percapita', 'log_ICT_exports',
       'log_Scientific_and_Technical_Journal',
       'Total Defence Expenditure per capita',
       'Defence R&D Expenditure (millions)', 'average_attacks_standardiz

In [74]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 32 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   country                                          27 non-null     object 
 1   NATO Member                                      27 non-null     object 
 2   International Cooperation                        27 non-null     int64  
 3   Incident Response Capabilities                   27 non-null     int64  
 4   Cyber Crime                                      27 non-null     int64  
 5   Training and Education                           27 non-null     int64  
 6   CIIP                                             27 non-null     int64  
 7   Citizen's Awarness                               27 non-null     int64  
 8   Incident Report Mechanism                        27 non-null     int64  
 9   Baseline Security Requirements    

In [75]:
# For average_attacks_standardized column, replace NaN values with 0
merged_df['average_attacks_standardized'] = merged_df['average_attacks_standardized'].fillna(0)

## Global Cybersecurity Index 2020 ##

Source: https://www.itu.int/dms_pub/itu-d/opb/str/D-STR-GCI.01-2021-PDF-E.pdf

In [76]:
merged_df['country']

0            Austria
1            Belgium
2           Bulgaria
3            Croatia
4             Cyprus
5     Czech Republic
6            Denmark
7            Estonia
8            Finland
9             France
10           Germany
11            Greece
12           Hungary
13           Ireland
14             Italy
15            Latvia
16         Lithuania
17        Luxembourg
18             Malta
19       Netherlands
20            Poland
21          Portugal
22           Romania
23          Slovakia
24          Slovenia
25             Spain
26            Sweden
Name: country, dtype: object

In [77]:
# GCI 2020 dataset creation

# Dictionaries of EU countries and x
eu_countries = {
    'Austria': 93.89,
    'Belgium': 96.25,
    'Bulgaria': 67.38,
    'Croatia': 92.53,
    'Cyprus': 88.82,
    'Czech Republic': 74.37,
    'Denmark': 92.60,
    'Estonia': 99.48,
    'Finland': 92.07,
    'France': 97.50,
    'Germany': 97.41,
    'Greece': 93.98,
    'Hungary': 91.28,
    'Ireland': 85.86,
    'Italy': 96.13,
    'Latvia': 97.28,
    'Lithuania': 97.93,
    'Luxembourg': 97.41,
    'Malta': 83.65,
    'Netherlands': 97.05,
    'Poland': 93.86,
    'Portugal': 97.32,
    'Romania': 76.29,
    'Slovakia': 92.36,
    'Slovenia': 74.93,
    'Spain': 98.52,
    'Sweden': 94.59
}

# Create a new column with the values from the dictionary
merged_df['GCI 2020'] = merged_df['country'].map(eu_countries)

# Display the first five rows of merged_df
print(merged_df.head())


    country NATO Member  International Cooperation  \
0   Austria          No                          1   
1   Belgium         Yes                          1   
2  Bulgaria         Yes                          1   
3   Croatia         Yes                          1   
4    Cyprus         Yes                          1   

   Incident Response Capabilities  Cyber Crime  Training and Education  CIIP  \
0                               1            1                       1     1   
1                               1            1                       1     0   
2                               1            1                       1     0   
3                               1            1                       0     0   
4                               1            1                       1     1   

   Citizen's Awarness  Incident Report Mechanism  \
0                   0                          1   
1                   1                          1   
2                   1                 

## Analysis ##

In [78]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 33 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   country                                          27 non-null     object 
 1   NATO Member                                      27 non-null     object 
 2   International Cooperation                        27 non-null     int64  
 3   Incident Response Capabilities                   27 non-null     int64  
 4   Cyber Crime                                      27 non-null     int64  
 5   Training and Education                           27 non-null     int64  
 6   CIIP                                             27 non-null     int64  
 7   Citizen's Awarness                               27 non-null     int64  
 8   Incident Report Mechanism                        27 non-null     int64  
 9   Baseline Security Requirements    

In [79]:
del merged_df['Average number of cyber incidents']

KeyError: 'Average number of cyber incidents'

In [ ]:
# Convert all the float into integers
merged_df['Cybersecurity exercises'] = merged_df['Cybersecurity exercises'].astype(int)
merged_df['Incentives to private sector to invest'] = merged_df['Incentives to private sector to invest'].astype(int)

In [ ]:
# Check data types
print(merged_df.dtypes)

# Convert 'GCI 2020' to a numeric type (assuming it's a numeric variable)
merged_df['GCI 2020'] = pd.to_numeric(merged_df['GCI 2020'], errors='coerce')

# Check data types again
print(merged_df.dtypes)

country                                             object
NATO Member                                         object
International Cooperation                            int64
Incident Response Capabilities                       int64
Cyber Crime                                          int64
Training and Education                               int64
CIIP                                                 int64
Citizen's Awarness                                   int64
Incident Report Mechanism                            int64
Baseline Security Requirements                       int64
Cybersecurity exercises                              int64
R&D                                                  int64
National Cyber Contigency Plans                      int64
Cooperatioon between Public Agencies                 int64
Public-Private Partnership                           int64
Balance Security-Privacy                             int64
Information Security Standards                       int

In [ ]:
merged_df.head()

,country,NATO Member,International Cooperation,Incident Response Capabilities,Cyber Crime,Training and Education,CIIP,Citizen's Awarness,Incident Report Mechanism,Baseline Security Requirements,...,CSIRTs,Military CSIRTs,Cyber Exercises under CSDP,log_GDP_percapita,log_ICT_exports,log_Scientific_and_Technical_Journal,Total Defence Expenditure per capita,Defence R&D Expenditure (millions),average_attacks_standardized,GCI 2020
0,Austria,No,1,1,1,1,1,0,1,1,...,10,0,0,10.86,23.00,9.53,368,"7,5",0.647059,93.89
1,Belgium,Yes,1,1,1,1,0,1,1,0,...,11,0,1,10.81,23.61,9.74,468,"17,2",1.117647,96.25
2,Bulgaria,Yes,1,1,1,1,0,1,1,0,...,1,0,1,9.53,21.72,8.32,156,"4,8",0.647059,67.38
3,Croatia,Yes,1,1,1,0,0,0,1,1,...,2,0,1,9.82,20.98,8.48,296,"0,6",0.117647,92.53
4,Cyprus,Yes,1,1,1,1,1,1,0,0,...,3,0,2,10.35,22.26,7.30,509,"0,3",0.235294,88.82


In [ ]:
# Check for missing values in the dataframe
missing_values = merged_df.isnull().sum()

# Check for infinite values (inf) in the dataframe
infinite_values = merged_df.isin([np.inf, -np.inf]).sum()

# Print the columns with missing or infinite values
print("Columns with missing values:")
print(missing_values[missing_values > 0])

print("\nColumns with infinite values:")
print(infinite_values[infinite_values > 0])

# Handle missing or infinite values (e.g., by imputation or removal)


Columns with missing values:
Series([], dtype: int64)

Columns with infinite values:
Series([], dtype: int64)


In [ ]:
# Create a dictionary with country-value pairs for Defence R&D Expenditure
defence_rnd_values = {
    'Austria': 7.5,
    'Belgium': 17.2,
    'Bulgaria': 4.8,
    'Croatia': 0.6,
    'Cyprus': 0.3,
    'Czech Republic': 16.1,
    'Denmark': 1300.0,
    'Estonia': 5.1,
    'Finland': 46.7,
    'France': 6500.0,
    'Germany': 1995.259,
    'Greece': 22.7,
    'Hungary': 4.8,
    'Ireland': 0.3,
    'Italy': 61.5,
    'Latvia': 5.0,
    'Lithuania': 4.7,
    'Luxembourg': 0.7,
    'Malta': 0.0,
    'Poland': 148.0,
    'Portugal': 2.6,
    'Romania': 57.7,
    'Slovakia': 2.0,
    'Slovenia': 2.5,
    'Spain': 115.9,
    'Sweden': 88.2
}

# Replace NaN values in "Defence R&D Expenditure (millions)" with dictionary values
merged_df['Defence R&D Expenditure (millions)'] = merged_df['Defence R&D Expenditure (millions)'].fillna(merged_df['country'].map(defence_rnd_values))

# Check the updated DataFrame
print(merged_df)

           country NATO Member  International Cooperation  \
0          Austria          No                          1   
1          Belgium         Yes                          1   
2         Bulgaria         Yes                          1   
3          Croatia         Yes                          1   
4           Cyprus         Yes                          1   
5   Czech Republic         Yes                          1   
6          Denmark         Yes                          1   
7          Estonia         Yes                          1   
8          Finland         Yes                          1   
9           France         Yes                          1   
10         Germany         Yes                          1   
11          Greece         Yes                          1   
12         Hungary         Yes                          1   
13         Ireland         Yes                          1   
14           Italy         Yes                          1   
15          Latvia      

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Data columns (total 33 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   country                                          26 non-null     object 
 1   NATO Member                                      26 non-null     object 
 2   International Cooperation                        26 non-null     int64  
 3   Incident Response Capabilities                   26 non-null     int64  
 4   Cyber Crime                                      26 non-null     int64  
 5   Training and Education                           26 non-null     int64  
 6   CIIP                                             26 non-null     int64  
 7   Citizen's Awarness                               26 non-null     int64  
 8   Incident Report Mechanism                        26 non-null     int64  
 9   Baseline Security Requirements    

In [ ]:
# Convert float to a numeric type (assuming it's a numeric variable)
merged_df['Defence R&D Expenditure (millions)'] = pd.to_numeric(merged_df['Defence R&D Expenditure (millions)'], errors='coerce')
merged_df['log_ICT_exports'] = pd.to_numeric(merged_df['log_ICT_exports'], errors='coerce')
merged_df['log_GDP_percapita'] = pd.to_numeric(merged_df['log_GDP_percapita'], errors='coerce')
merged_df['log_Scientific_and_Technical_Journal'] = pd.to_numeric(merged_df['log_Scientific_and_Technical_Journal'], errors='coerce')
merged_df['Total Defence Expenditure per capita'] = pd.to_numeric(merged_df['Total Defence Expenditure per capita'], errors='coerce')
merged_df['GCI 2020'] = pd.to_numeric(merged_df['GCI 2020'], errors='coerce')
merged_df['average_attacks_standardized'] = pd.to_numeric(merged_df['average_attacks_standardized'], errors='coerce')

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Data columns (total 33 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   country                                          26 non-null     object 
 1   NATO Member                                      26 non-null     object 
 2   International Cooperation                        26 non-null     int64  
 3   Incident Response Capabilities                   26 non-null     int64  
 4   Cyber Crime                                      26 non-null     int64  
 5   Training and Education                           26 non-null     int64  
 6   CIIP                                             26 non-null     int64  
 7   Citizen's Awarness                               26 non-null     int64  
 8   Incident Report Mechanism                        26 non-null     int64  
 9   Baseline Security Requirements    

In [ ]:
import statsmodels.api as sm

# Assuming your dataset is stored in a DataFrame called df

# Define the dependent variable (Y) and explanatory variables (X)
Y = merged_df['GCI 2020']
X = merged_df[['CSIRTs', 'Military CSIRTs', 'R&D', 'CIIP',
               'Cyber Exercises under CSDP', 'Total Defence Expenditure per capita',
               'log_GDP_percapita', 'log_ICT_exports',
               'Defence R&D Expenditure (millions)',
               'average_attacks_standardized']]

# Add a constant term to the X matrix (intercept)
X = sm.add_constant(X)

# Fit the multivariate regression model
model = sm.OLS(Y, X).fit()

# Print the summary of the regression analysis
# Check for missing values in the dataframe
missing_values = merged_df.isnull().sum()

# Check for infinite values (inf) in the dataframe
infinite_values = merged_df.isin([np.inf, -np.inf]).sum()

# Print the columns with missing or infinite values
print("Columns with missing values:")
print(missing_values[missing_values > 0])

print("\nColumns with infinite values:")
print(infinite_values[infinite_values > 0])


MissingDataError: exog contains inf or nans

In [ ]:
# Check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create a dataframe that contains the VIFs of each explanatory variable
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

# View the dataframe
print(vif)

In [ ]:
# Correlation matrix
corr_matrix = merged_df.corr()

# Print the correlation matrix
print(corr_matrix)

In [ ]:
# Set a correlation threshold
correlation_threshold = 0.2  # For example, you can start with 0.2

# Create a boolean mask of variables to keep
variables_to_keep = np.abs(corr_matrix['GCI 2020']) >= correlation_threshold

variables_to_keep


In [ ]:
print(merged_df['country'])